In [4]:
using PyPlot
using JLD
using Dates
using DSP: conv

mr = include("../../Tools/Model_Reduction_Dev.jl")
at = include("../../Tools/AnalysisToolbox.jl")

server = startswith(pwd(), "/u5/jaredm") ? true : false
# When I want the standard lin et al. (2017) data.|
sol_file = server ? "../../../data/KSE_Data/KSE_sol_lin.jld" :
   "Examples/KSE/Data/KSE_sol_lin.jld"

"../../../data/KSE_Data/KSE_sol_lin.jld"

In [2]:
# Load Old Data
@time Data = load(sol_file)
print("Data Loaded")
uu = Data["dat_uu"];
vv = Data["dat_vv"];
tt = Data["dat_tt"];

obs_gap= Data["obs_gap"];
h  = Data["h"];
P  = Data["P"];
N  = Data["N"]

Δt = h*gap

  2.474415 seconds (4.37 M allocations: 1.293 GiB, 2.88% gc time)
Data Loaded

LoadError: UndefVarError: gap not defined

In [3]:
function InvBurgRK4_1step(x)
   lx = length(x)
   function F(x)
      𝑥 = [conj(reverse(x, dims = 1)); 0; x]
      -im / 2 * (2π / P * (1:lx) / N) .* conv(𝑥, 𝑥)[2*lx+2:3*lx+1]
   end

   Δt = h * obs_gap

   k1 = F(x)
   k2 = F(x .+ Δt * k1 / 2)
   k3 = F(x .+ Δt * k2 / 2)
   k4 = F(x .+ Δt * k3)
   A = @. x + Δt / 6 * (k1 + 2k2 + 2k3 + k4)
end

function Inertialman_part_short(x)
   lx = length(x)
   𝑥(j) = (j <= lx ? x[j] : im * sum(x[l] * x[j-l] for l = j-lx:lx))

   L = complex(zeros(binomial(lx + 1, 2)))
   i = 1
   for j = 1:lx
      for k = j:lx # k should normaly go from 1 to lx but i changed it to go from j to lx.
         L[i] = 𝑥(j + lx) * 𝑥(j + lx - k)
         i += 1
      end
   end
   L
end


Psi(x) = [x; InvBurgRK4_1step(x); Inertialman_part_short(x)]

Psi (generic function with 1 method)

In [6]:
sig = vv[:,2:10];
pred= mr.get_pred(vv[:,2:10],Psi);

LoadError: InexactError: Float64(-75.03733880411616 + 105.73807940821219im)

In [ ]:
pwd()

In [ ]:
lags = -4000:4000
let plt = plot
    for m = 2:6
        figure()
        A = at.my_autocor(vv[m,:],lags)
        plt(Δt*lags,A)
        legend()
        title("mode $m")
    end
end